# Classification on CIFAR and ImageNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/### Research in ML/Pitt-ML/MLPaperReading/Codebase/how-do-vits-work-transformer"

/content/drive/MyDrive/### Research in ML/Pitt-ML/MLPaperReading/Codebase/how-do-vits-work-transformer


In [ ]:
%ls

classification.ipynb  losslandscape-colab.ipynb  ops/        robustness.ipynb
configs/              losslandscape.ipynb        README.md   runs/
LICENSE               models/                    resources/


In [ ]:
!pip3 install -U einops timm PyYAML

     |████████████████████████████████| 431 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 72.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import time
import yaml
import copy
from pathlib import Path
import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import models
import ops.trains as trains
import ops.tests as tests
import ops.datasets as datasets
import ops.schedulers as schedulers

In [ ]:
# !pip3 install -U PyYAML

In [ ]:
# config_path = "configs/cifar10_general.yaml"
config_path = "configs/cifar100_general.yaml"
# config_path = "configs/imagenet_general.yaml"

with open(config_path) as f:
    args = yaml.load(f, Loader=yaml.FullLoader)
    print(args)

{'dataset': {'name': 'cifar100', 'root': '../data', 'mean': [0.5071, 0.4867, 0.4408], 'std': [0.2675, 0.2565, 0.2761], 'padding': 4}, 'train': {'warmup_epochs': 1, 'epochs': 200, 'batch_size': 128}, 'val': {'batch_size': 256, 'n_ff': 1}, 'model': {'stem': False, 'block': {}}, 'optim': {'name': 'SGD', 'lr': 0.1, 'momentum': 0.9, 'nesterov': False, 'weight_decay': 0.0005, 'scheduler': {'name': 'MultiStepLR', 'milestones': [60, 120, 160], 'gamma': 0.2}}, 'env': {}}


In [ ]:
dataset_args = copy.deepcopy(args).get("dataset")
train_args = copy.deepcopy(args).get("train")
val_args = copy.deepcopy(args).get("val")
model_args = copy.deepcopy(args).get("model")
optim_args = copy.deepcopy(args).get("optim")
env_args = copy.deepcopy(args).get("env")

In [ ]:
dataset_train, dataset_test = datasets.get_dataset(**dataset_args, download=True)
dataset_name = dataset_args["name"]
num_classes = len(dataset_train.classes)

dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=train_args.get("num_workers", 4), 
                           batch_size=train_args.get("batch_size", 128))
dataset_test = DataLoader(dataset_test, 
                          num_workers=val_args.get("num_workers", 4), 
                          batch_size=val_args.get("batch_size", 128))

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    num_classes
))

Files already downloaded and verified
Files already downloaded and verified
Train: 50000, Test: 10000, Classes: 100


## Model

In [ ]:
# AlexNet
# name = "alexnet_dnn"
# name = "alexnet_mcdo"
# name = "alexnet_dnn_smoothing"
# name = "alexnet_mcdo_smoothing"

# VGG
# name = "vgg_dnn_16"
# name = "vgg_mcdo_16"
# name = "vgg_dnn_smoothing_16"
# name = "vgg_mcdo_smoothing_16"

# Preact VGG
# name = "prevgg_dnn_16"
# name = "prevgg_mcdo_16"
# name = "prevgg_dnn_smoothing_16"
# name = "prevgg_mcdo_smoothing_16"

# ResNet
name = "resnet_dnn_18"
# name = "resnet_mcdo_18"
# name = "resnet_dnn_smoothing_18"
# name = "resnet_mcdo_smoothing_18"

# name = "resnet_dnn_50"
# name = "resnet_mcdo_50"
# name = "resnet_dnn_smoothing_50"
# name = "resnet_mcdo_smoothing_50"

# Preact ResNet
# name = "preresnet_dnn_50"
# name = "preresnet_mcdo_50"
# name = "preresnet_dnn_smoothing_50"
# name = "preresnet_mcdo_smoothing_50"

# ResNeXt
# name = "resnext_dnn_50"
# name = "resnext_mcdo_50"
# name = "resnext_dnn_smoothing_50"
# name = "resnext_mcdo_smoothing_50"

# WideResNet
# name = "wideresnet_dnn_50"
# name = "wideresnet_mcdo_50"
# name = "wideresnet_dnn_smoothing_50"
# name = "wideresnet_mcdo_smoothing_50"

# SEResNet
# name = "seresnet_dnn_18"
# name = "seresnet_mcdo_18"
# name = "seresnet_dnn_smoothing_18"
# name = "seresnet_mcdo_smoothing_18"

# name = "seresnet_dnn_50"
# name = "seresnet_mcdo_50"
# name = "seresnet_dnn_smoothing_50"
# name = "seresnet_mcdo_smoothing_50"

# ViT
# name = "vit_ti"
# name = "vit_s"
# name = "vit_b"
# name = "vit_l"
# name = "vit_h"

# PiT
# name = "pit_ti"
# name = "pit_xs"
# name = "pit_s"
# name = "pit_b"

# Mixer
# name = "mixer_ti"
# name = "mixer_s"
# name = "mixer_b"
# name = "mixer_l"
# name = "mixer_h"

vit_kwargs = {
    "image_size": 32, 
    "patch_size": 2,
}

model = models.get_model(name, num_classes=num_classes, 
                         stem=model_args.get("stem", False), **vit_kwargs)
# models.load(model, dataset_name, uid=current_time)

model: resnet_dnn_18 , params: 11.2M, output: [3, 100]


Parallelize the given `moodel` by splitting the input:

In [ ]:
name = model.name
model = nn.DataParallel(model)
model.name = name

## Train

Define a TensorBoard writer:

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

with open("%s/config.yaml" % log_dir, "w") as f:
    yaml.dump(args, f)
with open("%s/model.log" % log_dir, "w") as f:
    f.write(repr(model))

print("Create TensorBoard log dir: ", log_dir)

Create TensorBoard log dir:  runs/cifar100/resnet_dnn_18/20220217_190230


Train the model:

In [ ]:
gpu = torch.cuda.is_available()
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * train_args.get("warmup_epochs", 0))

trains.train(model, optimizer,
             dataset_train, dataset_test,
             train_scheduler, warmup_scheduler,
             train_args, val_args, gpu,
             writer, 
             snapshot=-1, dataset_name=dataset_name, uid=current_time)  # Set `snapshot=N` to save snapshots every N epochs.

(45.91 sec/epoch) Warmup epoch: 0, Loss: 3.9195, lr: 1.000e-01
NLL: 3.7436, Cutoffs: 0.0 %, 90.0 %, Accs: 12.780 %, 78.571 %, Uncs: 0.000 %, 99.966 %, IoUs: 5.490 %, 45.833 %, Freqs: 100.000 %, 0.140 %, Top-5: 36.280 %, Brier: 0.960, ECE: 6.923 %, ECE±: 6.923 %
The model is warmed up: 50.29 sec 

(43.68 sec/epoch) Epoch: 0, Loss: 3.1340, lr: 1.000e-01
NLL: 2.8378, Cutoffs: 0.0 %, 90.0 %, Accs: 27.550 %, 86.127 %, Uncs: 0.000 %, 99.669 %, IoUs: 15.618 %, 41.540 %, Freqs: 100.000 %, 1.730 %, Top-5: 59.590 %, Brier: 0.848, ECE: 3.899 %, ECE±: 3.899 %
(43.68 sec/epoch) Epoch: 1, Loss: 2.5456, lr: 1.000e-01
NLL: 2.4521, Cutoffs: 0.0 %, 90.0 %, Accs: 35.860 %, 92.508 %, Uncs: 0.000 %, 99.283 %, IoUs: 22.316 %, 62.549 %, Freqs: 100.000 %, 6.140 %, Top-5: 68.750 %, Brier: 0.783, ECE: 6.691 %, ECE±: 6.691 %
(43.60 sec/epoch) Epoch: 2, Loss: 2.1742, lr: 1.000e-01
NLL: 2.5205, Cutoffs: 0.0 %, 90.0 %, Accs: 35.750 %, 88.074 %, Uncs: 0.000 %, 98.304 %, IoUs: 22.952 %, 57.350 %, Freqs: 100.000 %, 9.

Save the model:

In [ ]:
models.save(model, dataset_name, current_time, optimizer=optimizer)

## Test

In [ ]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)

N: 1, NLL: 0.9016, Cutoffs: 0.0 %, 90.0 %, Accs: 77.620 %, 94.667 %, Uncs: 0.000 %, 85.299 %, IoUs: 64.530 %, 89.047 %, Freqs: 100.000 %, 61.690 %, Top-5: 93.940 %, Brier: 0.317, ECE: 4.555 %, ECE±: 3.892 %


In [ ]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1, 2, 3, 4, 5, 10, 20, 50]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)

N: 1, NLL: 0.9016, Cutoffs: 0.0 %, 90.0 %, Accs: 77.620 %, 94.667 %, Uncs: 0.000 %, 85.299 %, IoUs: 64.530 %, 89.047 %, Freqs: 100.000 %, 61.690 %, Top-5: 93.940 %, Brier: 0.317, ECE: 4.555 %, ECE±: 3.892 %
N: 2, NLL: 0.9016, Cutoffs: 0.0 %, 90.0 %, Accs: 77.620 %, 94.667 %, Uncs: 0.000 %, 85.299 %, IoUs: 64.530 %, 89.047 %, Freqs: 100.000 %, 61.690 %, Top-5: 93.940 %, Brier: 0.317, ECE: 4.555 %, ECE±: 3.892 %
N: 3, NLL: 0.9016, Cutoffs: 0.0 %, 90.0 %, Accs: 77.620 %, 94.667 %, Uncs: 0.000 %, 85.299 %, IoUs: 64.530 %, 89.047 %, Freqs: 100.000 %, 61.690 %, Top-5: 93.940 %, Brier: 0.317, ECE: 4.555 %, ECE±: 3.892 %
N: 4, NLL: 0.9016, Cutoffs: 0.0 %, 90.0 %, Accs: 77.620 %, 94.667 %, Uncs: 0.000 %, 85.299 %, IoUs: 64.530 %, 89.047 %, Freqs: 100.000 %, 61.690 %, Top-5: 93.940 %, Brier: 0.317, ECE: 4.555 %, ECE±: 3.892 %
N: 5, NLL: 0.9016, Cutoffs: 0.0 %, 90.0 %, Accs: 77.620 %, 94.667 %, Uncs: 0.000 %, 85.299 %, IoUs: 64.530 %, 89.047 %, Freqs: 100.000 %, 61.690 %, Top-5: 93.940 %, Brier: 